In [3]:
import elasticsearch
import os
import re
from elasticsearch_dsl import Search

es = elasticsearch.Elasticsearch()  # use default of localhost, port 9200

articles = []
titles = []
for root, dirs, files in os.walk('nyt/articles'):
    for file in files:
        if file[0] != '.':
            with open(os.path.join(root,file), 'r') as f:
                articles.append(f.readlines())
                titles.append(file)

for index,article in enumerate(articles):
    es.index(index='scrapped', doc_type='article', id=(index+1), body={
        'body': article[2:],
        'title': re.sub("^[0-9]+-","",titles[index]),
        'url': article[1],
        'authors': article[0].split(', ')
    })

print('Searching for articles containing a word in their body:\n')
response = Search(using=es, index='scrapped').query('match', body='Dortmund').execute()
for hit in response:
    print('\t' + hit.title)
    print('\n')

print('Searching for articles containing a word in their title:\n')
response = Search(using=es, index='scrapped').query('match', title='Paris').execute()
for hit in response:
    print('\t' + hit.title)
    print('\n')
    
print('Searching for articles written by a specific author:\n')
response = Search(using=es, index='scrapped').query('match', authors='Lilia Blaise').execute()
for hit in response:
    print('\t' + hit.title)
    print('\n')

print('Fuzzy Query:\n')
response = es.search(
    index="scrapped",
    body={
      "query": {
        "fuzzy" : {
            "body" : {
                "value" :"Dortmond"
            }
        }
    }
    }
)
for hit in response['hits']['hits']:
    print('\t'+hit['_source']['title'])
    print('\n')
    
print('Search for a query on two fields:\n')
response = Search(using=es, index='scrapped').query('match', authors='Lilia Blaise')\
    .query("match", title='France').execute()
for hit in response:
    print('\t' + hit.title)
    print('\n')

Searching for articles containing a word in their body:

	Culture Shock in the Promised Land of Germany


Searching for articles containing a word in their title:

	Paris: Un an après


	Victims of the Terror Attacks in Paris


	Paris: One Year On


	Paris Police Clear Out Migrant Camp and Destroy Tents


	Inquiry Finds Mounting Proof of Syria Link to Paris Attacks


	France Unsure if Raid Killed Top Suspect in Paris Attacks


	ISIS Militant Returned to Belgium Before Paris Attacks


	Two Paris Attackers Planned Another Assault, Prosecutor Says


	School’s Warnings About Paris Attacker Were Not Passed On


	ISIS Claims Responsibility, Calling Paris Attacks ‘First of the Storm’


Searching for articles written by a specific author:

	Father Killed in Turkey Attacks Was Trying to Save Son From ISIS


	Belgian Police Name Man Suspected of Being Salah Abdeslam’s Accomplice


	France Restricts ‘Salafistes,’ Film on Islamic Radicals


	France Unsure if Raid Killed Top Suspect in Paris Attack